In [1]:
# !pip install regex, pandas numpy, os, re

In [2]:
import pandas as pd
import numpy as np
import re
import regex
import os
import pandas.io.formats.excel as fmt_xl     # version >= 0.20
pd.options.display.max_rows=99
pd.options.display.max_columns=999

### ТЗ -> Разложить очищенные данные на:
1. Населенный пункт(это может быть и город и СНТ)
2. Улица в этом населенном пункте
3. Дом с номером

### 1. загружаем данные

In [3]:
df_1 = pd.read_csv('./data/Address_IJS/data_1501.csv', 
                #    encoding='utf-8'
                on_bad_lines='skip',
                sep=';',
                header=None,
                   )
df_1.head()

,0,1,2,3,4,5,6,7,8
0,2020,8,92401000000,NaN,16:16:214001:2776,Казань,NaN,NaN,NaN
1,2021,4,92401000000,NaN,16:50:210102:1657,"г. Казань, Авиастроительный р-н, снт ""Нарцисс""...","г. Казань, Авиастроительный р-н","снт ""Нарцисс""",16
2,2020,12,92248000009,NaN,16:33:140406:13432,"Пестречинский р-н, Бог.ское сп, дер. Куюки, у...",Куюки,Тополиная,34
3,2020,4,92234000000,NaN,16:24:140602:91,Орловское,тер ТСН СНТ Троицкий,NaN,15
4,2020,4,92234000000,9.263448e+10,16:24:150106:988,Столбищенское,Столбище,Лесная,3А


### 2. удаляем бинарные сообщения

In [4]:
df_1[5] = df_1[5].replace(u'\xa0', ' ', regex=True)

In [5]:
df_1.iloc[13083:13085, 5].to_list()

['Пестречинский р-н, Богородское СП, д Куюки, ул. 25-й Квартал, д 83/2',
 'Пестречинский р-н, Богородское СП, д Куюки, ул 25-й Квартал, дом 71/1']

### 3. составляем списки и словари для замен

In [6]:
cyr_alp = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
# cyr_alp
noise_words = [
    'Республика Татарстан', 
    # 'Татарстан',
    'Реублика', 'Республика', 'РеСПублика'
    # 'МО ', 
    # 'муниципальный', 
    # 'мууниципальный', 
    # 'район', 
    # 'р-н',
    # 'г.ской', 
    # 'город',
    # 'округ', 
    # 'сп',
    'Российская Федерация.',
    'Российская Федерация',
    'территория',
    'тер ',
    'тер. ',
    ]
change_words = {
    'муниципальный':'М',
    'иуниципальный':'М',
    'мууниципальный':'М',
    'Мунииципальное':'М',
    'округ':'О',
    'район':'МО',
    'м.р-н':'МО',
    'р-н':'МО',
    'г.ской':'М',
    'город':'г.',
    'г.ород':'г.',
    'ородг':'г.',
    'пгт':'пгт'.upper(),
    'МО г. Казань':'г. Казань',
    'МО г Казань':'г. Казань',
    'сел.поселение': 'СП',
    'сельское поселение': 'СП',
    'с/п':'СП',
    'сп.':'СП',
    'с.п.':'СП',
    'сп ':'СП',
    'Татарстана':'@@ ул. Татарстана',
    'Тaтарстан':'',
    # 'участок':'участок',
    'уч.':'участок',
    'з/у':'участок',
    'Бог.ское':'Богородское',
    'Бог.ское ':'Богородское',
    'гор. поселение': 'ПГТ',
}
snt_var = [
    'сад неком т-во', 
    'сад т-во', 
    'сдт', 
    'с.н.т.', 
    'СНТ'.lower(),
    'ДНТ',
    'ДНТ'.lower(),
    'ДНП',
    'ДНП'.lower(),
    'потребительский кооператив (некоммерческого) садоводческого товарищества',
    'потребительский кооператив',
    'садоводческого товарищества',
    'садоводческое товарищество',
    'сад потреб кооп',
    'с/т',
    ]
COND_VILLAGE = [
    r"пос\.", r"дер\.", 
    r"\д\.\s\\D+", r"\д\.\\D+", r"\с\\D+", 
    r"с\.", r"п\.", r" c\."
    ]
# prior_dict = [
#     (0, 'г.'),
#     (1, 'дер.'), 
#     (2, 'снт'), 
#     (3, 'ул.'), 
#     (4, r'д\.\s\\d+'), 
#     (5, r'д\.\s\s\d+'), 
#     (6, r'д\.\\d+'), 
#     (7, 'д.')
# ]
# prior_dict
prior_dict = {
    'г.':0,
    'дер.':1,
    r'c\.\s':2,
    r'\\sc\.\s':3,
    r'c\s':4,
    'СНТ':5, 
    'ул.':6,
    r'ул\.\s':7,
    r'д\.\s\\d+':8, 
    r'д\.\s\s\d+':9, 
    r'д\.\\d+':10,
    r'д\\d+':11,
    'д.':12,
    r'\w+$':13,
    r'\s\w+$':14,
}
# prior_dict
prior_rules = [
    r'г\. ',
    r'г ',
    'ПГТ', 
    r'п. ',
    'квартал',
    'квартал ',
    r'дер\.', 
    r' д ',
    r'пос\.', 
    r'сел\.', 
    r'село ',
    'cело',
    r'с\.', 
    r' с ', 
    'СНТ', 
    'ул\.', 'ул ', 
    r'д\.',
    r'дом ',
    ]

### 3.1. Проводим очистку данных в строках и столбцах ДС
как можно заметить, основной метод для очистки от мусора будет метод:

3.1.1 replace - заменяет исх. на указанное значение\
3.1.2 strip - удаляется пробелы справа-слева, если не указан параметр для удаления.

In [7]:
adresses_df = df_1.iloc[:,5:6].iloc[:, :]
adresses_df.columns = ['Adress']
df = adresses_df.copy()
df = df.apply(lambda x: x.str.strip(), axis=0)
df = df.apply(lambda x: x.str.strip(), axis=1)
# df = adresses_df['Adress'].str.split(',', expand=True).replace(pd.NA, '')

df = df.apply(lambda x: x.replace('Сад №', 'Сад_номер', regex=True))
df = df.apply(lambda x: x.replace('№', '', regex=True))

# df = df.replace(',', ' ', regex=True)
df = df.replace('  ', ' ', regex=True)
df = df.replace('»', '', regex=True)
df = df.replace('«', '', regex=True)
df = df.replace('"', '', regex=True)
df = df.apply(lambda x: x.str.strip(), axis=0)
df = df.apply(lambda x: x.str.strip(), axis=1)
df = df.fillna('-')
df = df.apply(lambda x: x.replace(noise_words, '', regex=True), axis=0)
df = df.apply(lambda x: x.replace(change_words, regex=True))
df = df.apply(lambda x: x.replace(snt_var, 'СНТ', regex=True))
df = df.replace('М О', 'МО', regex=False)
df = df.replace('улул.', 'ул.', regex=False)

In [8]:
df[df['Adress'].str.contains('дом ', regex=True)].head()

,Adress
1220,", М МО Кукморский, СП Большекукморское, село Б..."
7296,"г Казань, сад неком тов-во Садовод 11, дом 139"
12407,"г. Казань, СНТ Чайка , дом 45"
12700,"Пестречинский МО, Бог.ское СП, д Куюки, ул. Ве..."
12710,"Пестречинский МО, Бог.ское СП, д Куюки, ул. 2-..."


ниже ячейки с проверкой содержимого после преобразования. 

они носят справочно-информационный характер

In [9]:
df_1.iloc[1220:1221,5].to_list()

['Республика Татарстан, муниципальный район Кукморский, сельское поселение Большекукморское, село Большой Кукмор, улица Саид-Галеева, дом 2']

In [10]:
df.iloc[1220:1221,0].to_list()

[', М МО Кукморский, СП Большекукморское, село Большой Кукмор, улица Саид-Галеева, дом 2']

In [11]:
df.iloc[64:65,:].apply(lambda x: x.replace(r"д\s", r' % дер. ', regex=True))['Adress'].to_list()

['Высокогорский МО, Суксинское СП,  % дер. Кундурла, ул. Зеленая, 4А']

In [12]:
df.iloc[64:65,:].apply(lambda x: x.replace(r"ул.\s", r' @@ ул. ', regex=True))['Adress'].to_list()

['Высокогорский МО, Суксинское СП, д Кундурла,  @@ ул. Зеленая, 4А']

In [13]:
df.iloc[1:2,0].to_list()[0].strip().split(',')[-1].split(' ')

['', 'д.', '16']

расстявляем приоритеты для разбиения строки адреса

In [14]:
# сначала выделяем тип населенного пункта
priority_1 = [
    r"г\.\s\D",
    r"г\.\D",
    r"МО\s",
    r"ПГТ\s",
    # r"ПГТ\s".lower(),
    r'п\.\D',
    r'п\.\s\D',
    r'п\s\D',
    r'п\.\d',
    r'п\.\s\d',
    r"д\s\D",
    r"д.\s\D",
    r"пос\.",
    r"сел\.",
    r"село ",
    r"cело",
    r"с\.",
    r" с ",
    r"снт",
    r"СНТ",
    ]
# затем удица
priority_2 = [
    r'ул\.\d',
    r'ул\.\s\d',
    r'ул\.\s\D',
    r'ул\.\D',
    r'ул\s\D',
    r'ул\D',
]
# затем дом
priority_3 = [
    r'д\.\d',
    r'д\.\s\d',
    r'дом\s\d',
]
# словарь замен сокращений и замен на спец символы для определения приоритета
priority = {
    r'г\. ' : r' % г.',
    r'г\s': r' % г ',
    'МО': r' % МО',
    'ПГТ': r' % ПГТ', 
    r'п\. ': r' %п.',
    'квартал': r' % квартал',
    'квартал ': r' % квартал ',
    r'дер\.': r' % дер.', 
    r'д ': r' % д ',
    r'пос\.': r' %',
    r'сел\.': r' %',
    r'село ': r' %',
    'cело': r' %',
    r'с\.': r' %',
    r' с ': r' %',
    'снт': r' % СНТ',
    'СНТ': r' % СНТ',
    r'ул\.': r' @@ ул.',
    r'ул ': r' @@ ул ',
    r'д\.': r' ### д.',
    r'д ': r' ### д ',
    r'дом ': r' ### дом ',
}

In [15]:
priority.keys()

dict_keys(['г\\. ', 'г\\s', 'МО', 'ПГТ', 'п\\. ', 'квартал', 'квартал ', 'дер\\.', 'д ', 'пос\\.', 'сел\\.', 'село ', 'cело', 'с\\.', ' с ', 'снт', 'СНТ', 'ул\\.', 'ул ', 'д\\.', 'дом '])

3. Проводим следующую очистку и преобразование содержания строк адреса

In [16]:
# df['Adress'] = df.replace(r"г\.\D", r' %г. ', regex=True)
df['Adress'] = df.replace(r'МО ', r' %МО ', regex=True)
# df['Adress'] = df.replace(r'МО ', r' %МО ', regex=True)
df['Adress'] = df.replace(r'ПГТ ', r' %ПГТ ', regex=True)
df['Adress'] = df.replace(r"пос ", r' %пос. ', regex=True)
df['Adress'] = df.replace(r'п\. ', r' %пос. ', regex=True)
# df['Adress'] = df.replace(r'п\. ', r' %пос. ', regex=True)
# df['Adress'] = df.replace(r"п\s\D", r' %пос. ', regex=True)
# df['Adress'] = df.replace(r"п\s\d", r' %пос. ', regex=True)
# df['Adress'] = df.replace(r'пос\. ', r' %пос. ', regex=True)
df['Adress'] = df.replace(r"[^%]пос\. ", r' %пос. ', regex=True)
# df['Adress'] = df.replace(r"пос\s\d", r' %пос. ', regex=True)
df['Adress'] = df.replace(r'дер\. ', r' %дер. ', regex=True)
df['Adress'] = df.replace(r'дер ', r' %дер. ', regex=True)
df['Adress'] = df.replace(r'село ', r' %село ', regex=True)
# df['Adress'] = df.replace(r'с\. [A-ZА-Я]', r' %село ', regex=True)
# df['Adress'] = df.replace(r'с [A-ZА-Я]', r' %село ', regex=True)
# df['Adress'] = df.replace(r'сел\. [A-ZА-Я]', r' %село ', regex=True)
# df['Adress'] = df.replace(r'сел\.[A-ZА-Я]', r' %село ', regex=True)
# df['Adress'] = df.replace(r'сел [A-ZА-Я]', r' %село ', regex=True)
df['Adress'] = df.replace(r'СНТ', r' &&СНТ ', regex=True)
df['Adress'] = df['Adress'].replace(r"участок", '###участок', regex=True, limit=1)

In [17]:
df['Adress'] = df['Adress'].str.replace('участоксток', 'участок', regex=True)

Так выглядит исходная строка

In [18]:
df.iloc[333:334,:]['Adress'].to_list()

['г. Казань,  &&СНТ  Сад_номер2 ПО Элекон, ###участок 43']

Так выглядит преобразованная

In [19]:
df_1.iloc[333:334,5].to_list()

['г. Казань,  сдт "Сад №2 ПО Элекон", участок 43']

In [20]:
# данный метод будет с помощью библиотеки re искать нужные блоки
def check_pattern(ptrn, string):
    return len(re.findall(ptrn, string))

In [21]:
# ниже методы очистки и преобразования для зашумленных данных
# то есть надо понять, если в адресе нет дома, а есть только участок (СНТ и др.)
def clean_row(row, pat, change, sign='%'):
    # print(type(row))
    row = row.lstrip('%')
    if len(row.split(' '))==1:
        return row
    if row.count(sign)>0:
        row_1 = row.split(sign)[1]
        row_0 = row.split(sign)[0]
    else:
        row_0 = ''
        row_1 = row
    if re.search(pat, row_1) is not None:
        pos_s = re.search(pat, row_1).start()
        pos_e = re.search(pat, row_1).end()
        # print(pos_s, pos_e)
        if pos_e - pos_s > 1:
            return row_0 + row_1.replace(row_1[pos_s:pos_e - 1], change)
        else:
            return row_0 + sign + row_1.replace(row_1[pos_s:pos_e], change)
    else:
        return row_0 + sign + row_1

def check_row_house(s, ptr=r'д '):
    # print(str(s))
    # row = s.to_list()[0]
    row = s
    row = row.lstrip('%')
    try:
        # print(s.split(' '))
        if len(s.split(' '))==1:
            return s
        if row.count('%')>1:
            lst = row.split('%')
            row_0 = lst[0] + '%'
            row_1 = lst[1]
        # elif row.count('%')==1:
        #     lst = row.split('%')
        #     row_0 = lst[0] + '%'
        #     row_1 = lst[1]
        else:
            row_0 = ''
            row_1 = row
        # print(row_0)
        # print(row_1)
        try:
            pos_s, pos_e = re.search(ptr, row_1).span()
        except Exception as e:
            # print(e)
            return s
    
        if row_1[pos_e:pos_e+1] not in cyr_alp:
            # print('Digit:', row_1[pos_e])
            return row_0 + row_1[:pos_s] + row_1[pos_s:pos_e].replace(row_1[pos_s:pos_e-1], '###д.', 1) + row_1[pos_e:]
        # elif regex.search(r'\p{IsCyrillic}', s[pos_e:pos_e+1]):
        if row_1[pos_e:pos_e+1] in cyr_alp:
            # print('Letter:', row_1[pos_e])
            return row_0 + row_1[:pos_s] + row_1[pos_s:pos_e].replace(row_1[pos_s:pos_e-1], '%дер.', 1) + row_1[pos_e:]
    except Exception as e:
        print(s)
        
def check_row_fields(s, ptr=r'участок '):
    # print(str(s))
    # row = s.to_list()[0]
    row = s
    row = row.lstrip('%')
    try:
        # print(s.split(' '))
        if len(s.split(' '))==1:
            return s
        if row.count('%')>1:
            lst = row.split('%')
            row_0 = lst[0] + '%'
            row_1 = lst[1]
        # elif row.count('%')==1:
        #     lst = row.split('%')
        #     row_0 = lst[0] + '%'
        #     row_1 = lst[1]
        else:
            row_0 = ''
            row_1 = row
        # print(row_0)
        # print(row_1)
        try:
            pos_s, pos_e = re.search(ptr, row_1).span()
        except Exception as e:
            # print(e)
            return s
    
        if row_1[pos_e:pos_e+1] in cyr_alp:
            # print('Digit:', row_1[pos_e])
            return row_0 + row_1[:pos_s] + row_1[pos_s:pos_e].replace(row_1[pos_s:pos_e-1], '###участок', 1) + row_1[pos_e:]
        # elif regex.search(r'\p{IsCyrillic}', s[pos_e:pos_e+1]):
        # if row_1[pos_e:pos_e+1] in cyr_alp:
            # print('Letter:', row_1[pos_e])
            # return row_0 + row_1[:pos_s] + row_1[pos_s:pos_e].replace(row_1[pos_s:pos_e-1], '%дер.', 1) + row_1[pos_e:]
    except Exception as e:
        print(s)

Выполняем преобразование

In [22]:
df['Adress'] = df['Adress'].apply(clean_row, pat=r" г\.[A-ZА-Яа-я]", change=' %г ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r' с\. [A-ZА-Я]', change=' %село ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r' с\. [A-ZА-Я]', change=' %село ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r' сел\. [A-ZА-Я]', change=' %село ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r' сел [A-ZА-Я]', change=' %село ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r" д. \\s[A-ZА-Я]", change=' %дер. ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r"д. [A-ZА-Я]", change=' %дер. ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r"д [A-ZА-Я]", change=' %дер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r" д \\s[A-ZА-Я]", change=' %дер. ')
# df['Adress'] = df.replace(r'д. ', r' %дер. ', regex=True)
# df['Adress'] = df.replace(r'д ', r' %дер. ', regex=True)
df['Adress'] = df['Adress'].apply(clean_row, pat=r"ул [A-ZА-Яа-я]", change=' @@ул. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"ул [0-9]", change=' @@ул. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r" ул. \D", change=' @@ул. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r" ул. [0-9]", change=' @@ул. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"улица ", change=' @@ул. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"улица [0-9]", change=' @@ул. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"переулок [0-9]", change=' @@пер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"переулок [А-Я]", change=' @@пер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"пер [0-9]", change=' @@пер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"пер [А-Я]", change=' @@пер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"пер. [0-9]", change=' @@пер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"пер. [А-Я]", change=' @@пер. ')
df['Adress'] = df['Adress'].apply(clean_row, pat=r"аллея", change=' @@аллея ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r"д. \d", change=' ###д. ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r"д \d", change=' ###д. ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r"д \D", change=' % дер. ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r"д\. \D", change=' % дер. ')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r" д\.\s[0-9]", change=' ###д. ', sign='%')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r" д\s[0-9]", change=' ###д. ',  sign='%')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r" д. [0-9]", change=' ###д. ',  sign='%')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r" д.[0-9]", change=' ###д. ',  sign='%')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r" д [0-9]", change=' ###д. ',  sign='%')
# df['Adress'] = df['Adress'].apply(clean_row, pat=r" д[0-9]", change=' ###д. ',  sign='%')
df['Adress'] = df.replace("ул @@ул.", " @@ул. ", regex=True)
# df_1.iloc[10562:10563,5].apply(clean_row, pat=r"д\s[A-ZА-Яа-я]", change=' ###д. ',  sign='%').to_list()
# df_1.iloc[10562:10563,5].apply(clean_row, pat=r"д [^0-9]", change=' ###д.',  sign='%').to_list()
# df_1.iloc[10562:10563,5].apply(clean_row, pat=r"д [A-ZА-Яа-я]", change=' ###д.',  sign='%').to_list()
df['Adress'] = df['Adress'].apply(check_row_house)
# res
# str_ = df_1.iloc[10562:10563,5].to_list()[0].split('д')#.str.replace()#.to_list()
# str_
# check_row(res)

In [23]:
print(df.iloc[10562:10563,0].to_list())
check_row_house(df.iloc[10562:10563,0].apply(check_row_house, ptr='д ').to_list()[0])

['Пестречинский МО, %дер. Куюки,  @@ул. Песочная, д 30,']


'Пестречинский МО, %дер. Куюки,  @@ул. Песочная, ###д. 30,'

In [24]:
print(df.iloc[42:43,0].to_list())
df.iloc[42:43,0].apply(check_row_house, ptr='д ').to_list()

['%Тукаевский МО, Мелекесское СП,  &&СНТ  Возрождение, @@ул. 23, 24, ###участок 1, 2']


['%Тукаевский МО, Мелекесское СП,  &&СНТ  Возрождение, @@ул. 23, 24, ###участок 1, 2']

Ниже пример непонятного адреса. То ли 197 это дом, то ли участок. Так как мы видим в содержании СНТ - будем считать, что это участок.

In [25]:
ptr="г. Казань; Авиастроительный МО;  СНТ  Ветеран КАПО им.С.П.Горбунова; 197"
df[df['Adress'].str.contains("СНТ  Ветеран КАПО им.С.П.Горбунова", regex=True)]

,Adress
104,"%г. Казань, Авиастроительный МО, &&СНТ Ветер..."
6932,"%г. Казань, г. Казань, Авиастроительный МО, &..."
10167,"%г. Казань, &&СНТ Ветеран КАПО им С.П.Горбу..."
10213,"г. Казань, %г Казань, Авиастроительный МО, Нек..."
11349,"%г. Казань, Авиастроительный МО, &&СНТ Ветер..."
13478,"%г Казань, &&СНТ Ветеран КАПО им.С.П.Горбуно..."
14270,"%г. Казань, Авиастроительный МО, некоммерческо..."
14288,"%г. Казань, г. Казань, Авиастроительный МО, &..."
16812,"%г. Казань, Авиастроительный МО, &&СНТ Ветер..."
18625,"% &&СНТ Ветеран КАПО им.С.П.Горбунова, д. 44"


In [26]:
# ищем номер дома в адресе
def house_number_find(s):
    last_elem = s.split(' ')[-1]
    if s.count('#') == 0:
        if (last_elem.isnumeric()) and (last_elem[0].isdigit()):
            if s.count('участок')>0:
                return " ".join(s.split(' ')[:-1]) + f"###участок {last_elem}"
            else:
                return " ".join(s.split(' ')[:-1]) + f"###д. {last_elem}"
        else:
            return s
    else:
        return s

Так в обработанном ДФ

In [27]:
df.iloc[2:3, 0].to_list()

['%Пестречинский МО, Богородское СП  дер. Куюки, @@ул. Тополиная, д. 34']

Так в исходном

In [28]:
df_1.iloc[2:3, 5].to_list()

['Пестречинский  р-н, Бог.ское сп, дер. Куюки, ул. Тополиная, д. 34']

Применяем метод

In [29]:
# house_number_find(df_1.iloc[104:105,5].to_list()[0])
# house_number_find(df.iloc[104:105,0].to_list()[0])
# house_number_find(df_1.iloc[105:106,5].to_list()[0])
# house_number_find(df.iloc[105:106,0].to_list()[0])
df['Adress'] = df['Adress'].apply(house_number_find)
df.head()

,Adress
0,Казань
1,"%г. Казань, Авиастроительный МО, &&СНТ Нарци..."
2,"%Пестречинский МО, Богородское СП дер. Куюки,..."
3,Орловское
4,Столбищенское


Видим что перед "д." появился спецсимвол ###

In [30]:
df.iloc[104:105,0].to_list()

['%г. Казань, Авиастроительный МО,  &&СНТ  Ветеран КАПО им.С.П.Горбунова,###д. 197']

В исходном нет его (естест-но)

In [31]:
df_1.iloc[632:633,5].to_list()

['г. Казань, Авиастроительный р-н,  жилой массив Северный, ул. Леонида Черкасова, д. 26']

In [32]:
df_house = df['Adress'].str.split("###", expand=True, n=1)
df_house.columns = ['adress', 'house']
df_house['house'].replace('###', '', regex=True, inplace=True)
# df_house['house'].replace('д.', '', regex=True, inplace=True)
df_house['house'] = df_house['house'].str.strip()
df_house['house'].fillna('-', inplace=True)
# df_house.head()
df_street = df_house['adress'].str.split('@@', expand=True, n=1)
df_street.columns = ['adress', 'street']
df_street = df_street.fillna('-')
df_street = df_street.apply(lambda x: x.str.strip(), axis=1)
df_street = df_street.apply(lambda x: x.str.strip(), axis=0)
df_street.head()

,adress,street
0,Казань,-
1,"%г. Казань, Авиастроительный МО, &&СНТ Нарци...",-
2,"%Пестречинский МО, Богородское СП дер. Куюки,","ул. Тополиная, д."
3,Орловское,-
4,Столбищенское,-


In [33]:
# метод поиска в содержимом цифр (так как в адресе может быть не только номер, но и символы А / и тд.)
def find_digits(row):
    pat = r"[0-9]"
    # print(type(row))
    if re.search(pat, row) is not None:
        pos_s = re.search(pat, row).start()
        pos_e = re.search(pat, row).end()
        # print(pos_s, pos_e)
        if pos_e - pos_s > 1:
            return row[pos_s:pos_e]
        else:
            return pd.NA
    else:
        return pd.NA

In [34]:
# df_street_house = df_street['street'].str.split(r'[0-9]', regex=True, n=1, expand=True)
# df_street_house = df_street['street'].apply(lambda x: pd.NA if not x[-1:].isdigit() else x)
# df_street_house = df_street['street'].str.split(' ', expand=True)
df_street_house = df_street['street'].str.split(',', expand=True)
df_street_house = df_street_house.fillna('-')
df_street_house.columns = ['sub_street', 'sub_house_1', 'sub_house_2', 'sub_house_3']
df_street_house['sub_house'] = df_street_house['sub_house_1'] + df_street_house['sub_house_2'] + df_street_house['sub_house_3']
df_street_house = df_street_house[['sub_street', 'sub_house']]
df_street_house = df_street_house.fillna('-')
df_street_house = df_street_house.replace('-', '', regex=True)
df_street_house.head()

,sub_street,sub_house
0,,
1,,
2,ул. Тополиная,д.
3,,
4,,


Выполняем финальную подготовку данных

In [35]:
df_final = pd.concat([df_street['adress'], df_street_house, df_house['house']], axis=1)
df_final['house_new'] = df_final['house'] + df_final['sub_house']
df_final = df_final.replace('-', '', regex=True, )
df_final = df_final[['adress', 'sub_street', 'house_new']]
df_final.columns = ['punkt', 'street', 'house']
df_final = df_final.replace('%', '', regex=True)
df_final = df_final.replace('@', '', regex=True)
df_final = df_final.replace('#', '', regex=True)
df_final = df_final.replace('&', '', regex=True)
df_final = df_final.replace(',', ';', regex=True)
df_final = df_final.replace(pd.NA, '', regex=True)
df_final = df_final.replace(pd.NA, '-')
df_final['house'] = df_final['house'].replace('"', '', regex=True)
df_final['house'] = df_final['house'].replace(';', '', regex=True)
df_final['house'] = df_final['house'].replace('дер.', '', regex=True)
df_final['street'] = df_final['street'].replace('ул.', '', regex=True)
df_final = df_final.apply(lambda x: x.str.strip(), axis=0)
df_final = df_final.apply(lambda x: x.str.strip(), axis=1)
df_final.head()

,punkt,street,house
0,Казань,,
1,г. Казань; Авиастроительный МО; СНТ Нарцисс; д.,,д. 16
2,Пестречинский МО; Богородское СП дер. Куюки;,Тополиная,д. 34 д.
3,Орловское,,
4,Столбищенское,,


In [36]:
df_final['punkt'] = df_final['punkt'].replace('', '-')
df_final['street'] = df_final['street'].replace('', '-')
df_final['house'] = df_final['house'].replace('', '-')
df_final['house'] = df_final['house'].str.strip()
df_final['house'] = df_final['house'].str.rstrip(";")
df_final['house'] = df_final['house'].str.lstrip(";")
df_final['street'] = df_final['street'].str.strip()
df_final['street'] = df_final['street'].str.rstrip(";")
df_final['street'] = df_final['street'].str.lstrip(";")
df_final['punkt'] = df_final['punkt'].str.strip()
df_final['punkt'] = df_final['punkt'].str.rstrip(";")
df_final['punkt'] = df_final['punkt'].str.lstrip(";")
df_final['punkt'] = df_final['punkt'].replace('участоксток', '', regex=True)
df_final['street'] = df_final['street'].replace('участоксток', '', regex=True)
df_final['house'] = df_final['house'].replace('участоксток', '', regex=True)
df_final['punkt'] = df_final['punkt'].replace('земельный', '', regex=True)
df_final['street'] = df_final['street'].replace('земельный', '', regex=True)
df_final['house'] = df_final['house'].replace('земельный', '', regex=True)
# df_final['punkt'] = df_final['punkt'].apply(lambda x: x if x[0]!=';' else x[1:])
# df_final['punkt'] = df_final['punkt'].apply(lambda x: x if x[-1]!=';' else x[:-1])

In [37]:
# df_final.to_excel("./output/test_1.xlsx")
cols_1 = ['punkt', 'street', 'dom']
cols_2 = ['input']

In [38]:
df_final['punkt'] = df_final['punkt'].str.strip()
df_final['street'] = df_final['street'].str.strip()
df_final['house'] = df_final['house'].str.strip()
df_final['house'] = df_final['house'].str.rstrip('д')
df_final['house'] = df_final['house'].str.lstrip('д')
df_final['house'] = df_final['house'].str.rstrip('д.')
df_final['house'] = df_final['house'].str.lstrip('д.')
df_final['street'] = df_final['street'].str.lstrip(' пер.')
df_final['street'] = df_final['street'].str.lstrip('пер.')
df_final['house'] = df_final['house'].str.strip()
df_final['house'] = df_final['house'].replace('участок', '', regex=True)
df_final.head()

,punkt,street,house
0,Казань,-,-
1,г. Казань; Авиастроительный МО; СНТ Нарцисс; д.,-,16
2,Пестречинский МО; Богородское СП дер. Куюки,Тополиная,34
3,Орловское,-,-
4,Столбищенское,-,-


In [39]:
df_final.head()

,punkt,street,house
0,Казань,-,-
1,г. Казань; Авиастроительный МО; СНТ Нарцисс; д.,-,16
2,Пестречинский МО; Богородское СП дер. Куюки,Тополиная,34
3,Орловское,-,-
4,Столбищенское,-,-


In [40]:
# записываем данные в формате excel
def save_excel_sheet(df, path, sheet_name, columns, index=False):
    df.columns = columns
    fmt_xl.header_style = columns
    df = df.replace('-', '')
    df = df.style.set_properties(**{
        # 'background-color': 'grey',
        'font-size': '9pt',
        'font-family': 'Arial',
    })
    # Create file if it does not exist
    if not os.path.exists(path):
        print('path not exists')
        df.to_excel(path, 
                    sheet_name=sheet_name, 
                    index=False,
                    columns=columns,
                    )
    else:
        # os.remove(path)
        # os.makedirs(path)
        with pd.ExcelWriter(path, engine='openpyxl', if_sheet_exists='replace', mode='a') as writer:
            df.to_excel(
                writer, 
                sheet_name=sheet_name,
                columns=columns,
                index=False)
# save_excel_sheet(df_final, path='./output/test_1.xlsx', sheet_name='Результат', columns=cols_1)
save_excel_sheet(df_final, path='./output/result.xlsx', sheet_name='Результат', columns=cols_1)
save_excel_sheet(df_1.iloc[:,5:6], path='./output/result.xlsx', sheet_name='Исходные', columns=cols_2)
# save_excel_sheet(df_1.iloc[:,5:6], path='./output/result.xlsx', sheet_name='Исходные', columns=cols_2)